This notebook demonstrates the procedure for training the multi-label models during the 2023 TRAM effort.

Only the annotations produced during this effort can be adapted for multi-label modeling. First, we will load the annotations.

In [1]:
import pandas as pd

data = pd.read_json('../data/tram2-data/multi_label.json')
all_labels = data['labels'].explode().dropna().unique()
data

,sentence,labels,doc_title
0,title: NotPetya Technical Analysis – A Triple ...,[],NotPetya Technical Analysis A Triple Threat F...
1,Executive Summary This technical analysis prov...,[],NotPetya Technical Analysis A Triple Threat F...
2,For more information on CrowdStrike’s proactiv...,[],NotPetya Technical Analysis A Triple Threat F...
3,NotPetya combines ransomware with the ability ...,[],NotPetya Technical Analysis A Triple Threat F...
4,It spreads to Microsoft Windows machines using...,[T1210],NotPetya Technical Analysis A Triple Threat F...
...,...,...,...
19173,[2] Eclypsium Blog - TrickBot Now Offers 'Tric...,[],AA21076A TrickBot Malware
19174,"Initial Version March 24, 2021:",[],AA21076A TrickBot Malware
19175,Added MITRE ATT&CK Technique T1592.003 used fo...,[],AA21076A TrickBot Malware
19176,Added new MITRE ATT&CKs and updated Table 1,[],AA21076A TrickBot Malware


In [ ]:
import transformers
import torch

mode: 'bert or gpt' = 'bert'
cuda = torch.device('cuda')

if mode == 'bert':
    model = transformers.BertForSequenceClassification.from_pretrained(
        "allenai/scibert_scivocab_uncased",
        num_labels=len(all_labels),
        output_attentions=False,
        output_hidden_states=False,
    )
    tokenizer = transformers.BertTokenizer.from_pretrained("allenai/scibert_scivocab_uncased", max_length=512)
elif mode == 'gpt':
    model = transformers.GPT2ForSequenceClassification.from_pretrained(
        "gpt2",
        num_labels=len(all_labels),
        output_attentions=False,
        output_hidden_states=False,
    )
    tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2", max_length=512)
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id
else:
    raise ValueError(f"mode must be one of bert or gpt, but is {mode = !r}")

model.train().to(cuda)

For single-label modeling, we represented the labels using one hot encoding. The representation here is similar, except each row can have more than one `1` if it represents a multi-label instance. Some rows will not have any `1`s if they represent a negative sample.

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer as MLB
from sklearn.model_selection import train_test_split

mlb = MLB()
mlb.fit([[c] for c in all_labels])

train, test = train_test_split(data, test_size=0.2, shuffle=True)

def load_data(x, y, batch_size=10):
    x_len, y_len = x.shape[0], y.shape[0]
    assert x_len == y_len
    for i in range(0, x_len, batch_size):
        slc = slice(i, i + batch_size)
        yield x[slc].to(cuda), y[slc].to(cuda)

def tokenize(instances: 'list[str]'):
    return tokenizer(instances, return_tensors='pt', padding='max_length', truncation=True, max_length=512).input_ids

def encode_labels(labels):
    """:labels: should be the `labels` column (a Series) of the DataFrame"""
    return torch.Tensor(mlb.transform(labels.to_numpy()))

In [4]:
x_train = tokenize(train['sentence'].tolist())
x_train

tensor([[  102,  7208,  4531,  ...,     0,     0,     0],
        [  102,   260, 24391,  ...,     0,     0,     0],
        [  102,  4975, 11554,  ...,     0,     0,     0],
        ...,
        [  102,   407, 14382,  ...,     0,     0,     0],
        [  102,   256,   165,  ...,     0,     0,     0],
        [  102,   111,  2057,  ...,     0,     0,     0]])

In [5]:
y_train = encode_labels(train['labels'])
y_train

tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
from statistics import mean

from tqdm import tqdm
from torch.optim import AdamW

optim = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

for epoch in range(6):
    epoch_losses = []
    for x, y in tqdm(load_data(x_train, y_train, batch_size=10)):
        model.zero_grad()
        out = model(x, attention_mask=x.ne(tokenizer.pad_token_id).to(int), labels=y)
        epoch_losses.append(out.loss.item())
        out.loss.backward()
        optim.step()
    print(f"epoch {epoch + 1} loss: {mean(epoch_losses)}")

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as calculate_score

model.eval()

x_test = tokenize(test['sentence'].tolist())

batch_size = 20
preds = []

with torch.no_grad():
    for i in range(0, x_test.shape[0], batch_size):
        x = x_test[i : i + batch_size].to(cuda)
        out = model(x, attention_mask=x.ne(tokenizer.pad_token_id).to(int))
        preds.extend(out.logits.to('cpu'))


In [ ]:
binary_preds = torch.vstack(preds).sigmoid().gt(.5).to(int)

predicted = pd.Series(mlb.inverse_transform(binary_preds)).apply(frozenset).reset_index(drop=True)
actual = test['labels'].apply(frozenset).reset_index(drop=True)
results = pd.concat({'preds': predicted, 'actual': actual}, axis=1)

results

While the formulae for precision, recall, and F1 are the same for multi-label evaluation, the procedure for counting true positives, false positives, and false negatives is not.

Where $P$ is the set of predicted labels for a given instance, and $A$ is the set of actual labels for the same instance, the true positives are the labels in $P \cap A$, the false positives are those in $P - A$, and the false negatives are those in $A - P$.

To give an example, if the actual labels for a sample are $\{a, c\}$, and the model predicts $\{c, d\}$, that is a true positive for $c$, a false positive for $d$, and a false negative for $a$.

In [ ]:
tp = results.apply((lambda r: r.preds & r.actual), axis=1).explode().value_counts()
fp = results.apply((lambda r: r.preds - r.actual), axis=1).explode().value_counts()
fn = results.apply((lambda r: r.actual - r.preds), axis=1).explode().value_counts()

support = actual.explode().value_counts().rename('#')

counts = pd.concat({'tp': tp, 'fp': fp, 'fn': fn}, axis=1).fillna(0).astype(int)

p = counts.tp.div(counts.tp + counts.fp).fillna(0)
r = counts.tp.div(counts.tp + counts.fn).fillna(0)
f1 = (2 * p * r) / (p + r)
scores = pd.concat({'P': p, 'R': r, 'F1': f1}, axis=1).fillna(0).sort_values(by='F1', ascending=False)

# calculate macro scores
scores.loc['(macro)'] = scores.mean()

# calculate micro scores
micro = counts.sum()
scores.loc['(micro)', 'P'] = mP = micro.tp / (micro.tp + micro.fp)
scores.loc['(micro)', 'R'] = mR = micro.tp / (micro.tp + micro.fn)
scores.loc['(micro)', 'F1'] = (2 * mP * mR) / (mP + mR)

scores.join(support)